# Analyze and Visualize NBA Team Ratings 

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/boxscores_team/"

# Net Ratings

In [ ]:
from nba_api.stats.endpoints import leaguedashteamstats

stats = leaguedashteamstats.LeagueDashTeamStats(
    measure_type_detailed_defense="Advanced"
)
df = stats.get_data_frames()[0]
df["Team"] = df["TEAM_NAME"]
cols = df.columns
df.columns = cols.str.replace("RATING", "RTG")
df = df.rename(columns={"OFF_RTG_RANK": "O_RANK", "DEF_RTG_RANK": "D_RANK"})

In [ ]:
df.columns

In [ ]:
df1 = df.sort_values(by="NET_RTG", ascending=False)
df1 = df1.reset_index(drop=True)
df1 = df1.reset_index(drop=False)
df1["#"] = df1["index"] + 1
df2 = df1[["#", "Team", "OFF_RTG", "O_RANK", "DEF_RTG", "D_RANK", "NET_RTG"]]
df = df2

In [ ]:
n_colors = 30
colors = px.colors.sample_colorscale(
    "Tropic", [n / (n_colors - 1) for n in range(n_colors)]
)
orank = df["O_RANK"] - 1
orank = orank.astype(np.int32).to_numpy()
carray_o = np.array(colors)[orank]
drank = df["D_RANK"] - 1
drank = drank.astype(np.int32).to_numpy()
carray_d = np.array(colors)[drank]
nrank = df["#"] - 1
nrank = nrank.astype(np.int32).to_numpy()
carray_n = np.array(colors)[nrank]
title = "NBA Net Rating Leaders 2023-24"
font = dict(color="black", family="Arial Black, monospace", size=11)
fig = go.Figure(
    data=[
        go.Table(
            columnwidth=[5, 50, 15, 15, 15, 15, 15],
            header=dict(
                values=list(df.columns),
                fill_color="snow",
                align=["center", "left", "center", "center"],
                font=dict(color="Black", family="Arial Black, monospace", size=12),
                line_color="grey",
            ),
            cells=dict(
                values=[
                    df["#"],
                    df["Team"],
                    df["OFF_RTG"],
                    df["O_RANK"],
                    df["DEF_RTG"],
                    df["D_RANK"],
                    df["NET_RTG"],
                ],
                fill_color=[
                    "white",
                    "white",
                    carray_o,
                    "white",
                    carray_d,
                    "white",
                    carray_n,
                ],
                align=["center", "left", "center", "center"],
                height=23,
                line_color="grey",  # lightgrey",
                font=font,
            ),
            # height=25
        ),
    ]
)
# fig.update_layout(title_text=title)
fig.update_layout(
    title=dict(
        text=title, y=0.99, x=0.1, font=dict(size=15, family="Arial Black, monospace")
    )
)
fig.add_annotation(x=0.0, y=0.0, text="@SravanNBA", showarrow=False, xshift=1, yshift=5)
fig.add_annotation(
    x=1.0,
    y=0.0,
    text="Source: nba.com/stats",
    showarrow=False,
    xshift=1,
    yshift=5,
    font=dict(size=10),
)
# fig.update_layout(title=dict(text=title,y=0.01,x=0.1,font=dict(size=15,family="Arial Black, monospace")))
tab_width = 550
tab_height = 770
fig.update_layout(width=tab_width, height=tab_height, margin=dict(t=25, b=1, l=1, r=1))
# fig.update_layout(autosize=True)
fig.show()
fig.write_image(fig_DIR + f"Teams_NET_RTG.png", scale=3)

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i df 
library(tidyverse)
library(gt)

df %>% 
  gt()%>%
  tab_header(
    title = md("**NBA Net Rating Leaders 2023-24**")) %>%
    data_color(columns = OFF_RTG, palette = c("red", "green")) %>%
    data_color(columns = DEF_RTG, palette = c("green","red")) %>%
    data_color(columns = NET_RTG, palette = c("red", "green")) %>%
    cols_align(align = "center",columns = c("#",OFF_RTG,O_RANK,DEF_RTG,D_RANK,NET_RTG))  %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_ratings/Teams_NET_RTG.png",zoom=5) 


# Net Rating Variance

In [ ]:
season = 2022
df1 = pd.read_csv(box_DIR + f"NBA_BoxScores_Adv_{season}.csv")
cols = [
    "gameId",
    "teamId",
    "offensiveRating",
    "defensiveRating",
    "netRating",
    "possessions",
]
df1 = df1[cols]
df1["Win"] = df1["netRating"] > 0
df1["Loss"] = df1["netRating"] < 0
stats = leaguegamelog.LeagueGameLog(
    player_or_team_abbreviation="T",
    season=season,
    season_type_all_star="Regular Season",
)
df2 = stats.get_data_frames()[0]

df2 = df2.rename(
    columns={
        "GAME_ID": "gameId",
        "TEAM_ID": "teamId",
        "TEAM_ABBREVIATION": "teamTricode",
        "GAME_DATE": "gameDate",
        "TEAM_NAME": "teamName",
    }
)
cols2 = ["gameId", "teamId", "teamTricode", "gameDate"]
df2 = df2[cols2]
df2["gameId"] = df2["gameId"].astype(int)
df3 = pd.merge(df2, df1, on=["gameId", "teamId"])
df4 = df3.sort_values(by=["teamTricode", "gameDate"]).reset_index(drop=True)
data = df4
data = data.rename(columns={"gameDate": "Date"})

In [ ]:
grp = data.groupby("teamTricode")["netRating"]
sorted_grp = grp.describe().sort_values(by="std", ascending=False).reset_index()
teams_sorted = sorted_grp["teamTricode"].tolist()
sorted_grp = sorted_grp.round(2)
sorted_grp.head()
sorted_grp = sorted_grp.set_index("teamTricode")
sorted_grp = sorted_grp.reset_index()

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i sorted_grp 
library(tidyverse)
library(gt)
df <- sorted_grp
df <- df %>% rename( "Team" = "teamTricode", "Games" = "count")
df %>% 
  gt()%>%
  tab_header(
    title = md("**NBA Team Net Rating Variance 2023-24**"),
    subtitle = md("Higher Std Deviation means more variance" )
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_spanner(
    label = "Percentiles",
    columns = c("25%", "50%", "75%")
    ) %>%
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_ratings/Teams_Variance.png",zoom=5) 


In [ ]:
teams_cat = pd.Categorical(data["teamTricode"], categories=teams_sorted)
df1 = data.assign(teams_cat=teams_cat)
df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
df = pd.merge(df1, df_teams)

In [ ]:
# df = df[df["teamTricode"]==team].reset_index()
var, y = "netRating", "Net Rating"
# var,y = "Minutes", "Minutes Played"
title = "NBA" + " " + y + " " + "Distribution"
p = (
    ggplot(df)  # What data to use
    + aes(x=var, group="teams_cat", fill="colorsTeam")  # What variable to use
    + geom_density(
        alpha=0.5,
        show_legend=False,
    )
    + scale_color_identity(aesthetics=["fill"])
    + facet_wrap(facets="~ teams_cat")
    + labs(
        x=y,
        y="Fraction of games",
        title=title,
        subtitle="x-axis shows the points and y-axis shows fraction of games with that Net Rating | sorted by highest std deviation to least",
        caption="@SravanNBA | source: nba.com/stats",
    )
    # + themes.theme_xkcd(base_size=14)
    + themes.theme_538(base_size=12)
    + theme(
        plot_title=element_text(face="bold", size=18),
        plot_subtitle=element_text(size=14),
        plot_margin=0.025,
    )
)
p.save(fig_DIR + "NBA" + "_" + var + ".png", dpi=300, width=15, height=10)
p.draw()

In [ ]:
var, y = "netRating", "Net Rating"
# var,y = "Minutes", "Minutes Played"
title = "NBA" + " " + y + " " + "Boxplot"
p = (
    ggplot(df)  # What data to use
    + aes(x="teams_cat", y=var, fill="colorsTeam")  # What variable to use
    # + geom_line()  # Geometric object to use for drawing
    + geom_boxplot(alpha=0.5)
    + scale_color_identity(aesthetics=["fill"])
    + scale_y_continuous(limits=[-20, 20], breaks=[-20, -10, 0, 10, 20])
    # + geom_hline(yintercept=0.752, linetype="dashed", color="blue", size=0.5)
    # + annotate("text", x=df["gameDate"].iloc[1], y=0.76, label="League Avg")
    + labs(x="Team", y=y, title=title, caption="@SravanNBA | source: nba.com/stats")
    # + themes.theme_xkcd(base_size=14)
    + themes.theme_538(base_size=12)
    + theme(plot_title=element_text(face="bold", size=18), plot_margin=0.025)
)
p.save(fig_DIR + "NBA" + "_" + var + "_boxplot.png", dpi=300, height=5, width=15)
p.draw()